# LOC Goes Lofi

This notebook demonstrates how you can create an audio mashup from random audio in the Web Archives Dot Gov audio collection.

In [ ]:
from pydub import AudioSegment
from pydub.playback import play
from glob import glob
import re
from random import choice
import os

In [ ]:
def speed_change(sound, speed=1.0):
    # https://stackoverflow.com/questions/51434897/how-to-change-audio-playback-speed-using-pydub
    # Manually override the frame_rate. This tells the computer how many
    # samples to play per second
    sound_with_altered_frame_rate = sound._spawn(sound.raw_data, overrides={
        "frame_rate": int(sound.frame_rate * speed)
    })

    # convert the sound with altered frame rate to a standard frame rate
    # so that regular playback programs will work right. They often only
    # know how to play audio at standard frame rate (like 44.1k)
    return sound_with_altered_frame_rate.set_frame_rate(sound.frame_rate)

def get_random_sound():
    # pydub supports mp3, wav, and mp4, so 
    # filter out extensions that are not those
    sounds = list(
        filter(
            lambda x: re.search(r'\.mp3|\.wav|\.mp4', x),
            [g for g in glob('lcwa_gov_audio_data/data/*')]))
    
    # get a random int from the range of 0 - number of items in sounds list
    n = choice(range(len(sounds)))
    print(sounds[n])
    
    # play up to 20 seconds of the sample
    play(AudioSegment.from_file(sounds[n])[:20000])    

In [ ]:
get_random_sound()

In [ ]:
# sampling sounds that were discovered from the get_random_sound function
silence = AudioSegment.silent(duration=1000)
opening_bit = AudioSegment\
.from_file('lcwa_gov_audio_data/data/VNHV4ETRBCPFRYMKTDQTBBXP7MDIRSES.mp3')[:10 * 300]
outro_speech = AudioSegment\
.from_file('lcwa_gov_audio_data/data/Q526WMMXREAFK6KIGZKC2LO42R2BRKUJ.mp3')[2000:11900]
elevator = AudioSegment.from_file('lcwa_gov_audio_data/data/CAQY57V3XBWV3R2MTF5OVIUAYMUSIMYZ.mp3')[:20000]
low_reverb = speed_change(
            AudioSegment.from_file('lcwa_gov_audio_data/data/EPOMMZ4E5SVGJ25AACNN34XCGC6FE4CK.mp3')[:700], .4
        )
thud = AudioSegment.from_file('lcwa_gov_audio_data/data/LKXPK4P7QM63MOQXGEDA42DAGKMLEI23.mp3')[600:900]
moon = AudioSegment.from_file('lcwa_gov_audio_data/data/KL43UGQ476FFG6YD66AVJSYMDHKE4AXL.wav')[:3000]
clap = AudioSegment.from_file('lcwa_gov_audio_data/data/UEMJ2NUMWD2EK74TWU22GTRLBQXXZ5GI.mp3')[:1500]
melody = AudioSegment.from_file('lcwa_gov_audio_data/data/UQKFDBADCQXSW7SY73FPKXNFPFAXQLVP.mp3')[1500:8500]
giant_bear = AudioSegment.from_file('lcwa_gov_audio_data/data/CD5OAIP5C5DV5K2G5DJKMXD5MMTT32AX.mp3')[1500:4700]
no = AudioSegment.from_file('lcwa_gov_audio_data/data/HXZRKQKIZZIE4D5KGT5BGDGSVOO57MIG.mp3')[:2000]
good_evening = AudioSegment.from_file('lcwa_gov_audio_data/data/KPE2N7CO5ELM2BR4B2CHV3OFZEXSJICT.mp3')[:3000]
low_level_background_noise = AudioSegment\
.from_file('lcwa_gov_audio_data/data/JLCIUWJD2BF4JMIBQSREMANR3SUSTJQA.mp4')[:10 * 2000]

In [ ]:
# combining and remixing some of the samples from above to create longer tracks
background = low_level_background_noise.overlay((silence*3).append(low_reverb*11))
moon_loop = moon[1000:3000]\
        .append(AudioSegment.silent(100))\
        .append(moon[2700:3000]*3)\
        .append(AudioSegment.silent(100))\
        .append(moon[1000:2000])\
        .append(AudioSegment.silent(100))\
        .append(moon[2600:3000])\
        .append(AudioSegment.silent(100))\
        .append(moon[2100:2400])\
        .append(AudioSegment.silent(100))\
        .append(moon[1000:3000])\
        .append(AudioSegment.silent(400))

In [ ]:
# piecing everything together into one track
final_sound = (((background).append(silence*40))
     .overlay(opening_bit.fade_in(1000)
              .append(silence*4)
              .append(good_evening)
              .append(silence)
              .append(moon_loop.fade_out(1000))
              .append(silence*4)
              .append(AudioSegment.silent(400))
              .append((giant_bear.append(silence*11))
                      .overlay((silence*3)
                               .append(speed_change(melody, .5)
                                       .fade_in(1000)
                                       .fade_out(1000)
                                      )))
              .append(no+10)
              .append(AudioSegment.silent(400))
              .append((clap.append(clap.reverse())))
              .append(silence)
              .append(outro_speech.fade_out(1000))
             ).overlay(silence
                       .append(speed_change(elevator, 1.6)
                               .reverse() * 2).append(silence*3)))                                  

In [ ]:
# saving track out with metadata
final_sound.export("loc_mashup.mp3", format="mp3", tags={
    'artist': 'Various artists',
    'creator': 'Chase Dooley',
    'album': 'LOC Mashup', 
    'comments': 'This is a composite of audio tracks taken from the LCWA .gov Audio dataset'})